In [13]:
# math, code系instruciton datasetの生成
import pandas as pd
import os
from datasets import load_dataset
import json
import random
from tqdm import tqdm
import re

data_folder="data/0523math"

#dataフォルダ内をリセット
os.system(f"mkdir {data_folder}")
os.system(f"rm -rf {data_folder}/*")


ds_dict={}


mkdir: cannot create directory ‘data/0523math’: File exists


In [14]:
# cleaning script
def clean_autogen(text):
    if text is None:
        return ""
    text=text.strip()
    return text

question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n\n### 応答:\n"


records=[]

# # mixtralで自動生成したQ&A

score_threshold=4
ng_words=[
          #回答を避けるプロンプトの削除
          "申し訳","分からない","分かりません","すみません",
          #図表などへの言及
          "図","表",
          "市","県",
          "漢字",
          ]
#日本関係の事項はハルシネーションが多い
ng_words+= [
    "北海", "青森", "岩手", "宮城", "秋田", "山形", "福島",
    "茨城", "栃木", "群馬", "埼玉", "千葉", "東京", "神奈川",
    "新潟", "富山", "石川", "福井", "山梨", "長野", "岐阜",
    "静岡", "愛知", "三重", "滋賀", "京都", "大阪", "兵庫",
    "奈良", "和歌山", "鳥取", "島根", "岡山", "広島", "山口",
    "徳島", "香川", "愛媛", "高知", "福岡", "佐賀", "長崎",
    "熊本", "大分", "宮崎", "鹿児島", "沖縄"
]
ng_words+= [
    "札幌", "仙台", "新潟", "横浜", "川崎", "相模原", "名古屋",
    "京都", "大阪", "神戸", "堺", "広島", "北九州", "福岡",
    "熊本", "那覇", "千葉", "さいたま", "静岡", "浜松", "岡山",
    "鹿児島", "長崎", "松山", "高松", "徳島", "高知", "大分",
    "宮崎", "長野", "金沢", "富山", "福井", "松本", "豊橋",
    "岐阜", "甲府", "奈良", "和歌山", "姫路", "福山", "呉",
    "倉敷", "宇部", "下関", "高岡", "今治", "東広島", "伊勢",
    "旭川", "函館", "室蘭", "帯広", "釧路", "青森", "八戸",
    "弘前", "秋田", "盛岡", "山形", "福島", "郡山", "いわき",
    "宇都宮", "前橋", "高崎", "水戸", "日立", "土浦", "柏",
    "松戸", "浦安", "市川", "船橋", "川口", "越谷", "草加",
    "所沢", "川越", "熊谷", "相馬", "大宮", "高槻", "豊中",
    "茨木", "八尾", "寝屋川", "枚方", "平塚", "藤沢", "厚木",
    "横須賀", "三浦", "鎌倉", "小田原", "逗子", "鎌倉", "茅ヶ崎",
    "三鷹", "八王子", "立川", "府中", "調布", "町田", "稲城",
    "横浜", "厚木", "大和", "綾瀬", "藤沢", "鎌倉", "平塚",
    "鶴岡", "酒田", "米沢", "会津若松", "白河", "前橋", "草津",
    "沼津", "富士", "清水", "掛川", "豊田", "岡崎", "一宮",
    "津", "四日市", "鈴鹿", "名張", "甲賀", "彦根", "大津",
    "舞鶴", "長浜", "高槻", "枚方", "東大阪", "西宮", "芦屋",
    "加古川", "明石", "洲本", "淡路", "津山", "玉野", "赤磐",
    "出雲", "松江", "大田", "浜田", "益田", "鳥取", "倉吉",
    "米子", "西条", "新居浜", "大洲", "八幡浜", "今治", "佐世保",
    "平戸", "諫早", "雲仙", "南島原", "宇和島", "八幡浜", "中津",
    "豊後高田", "別府", "佐伯", "延岡", "小林", "日向", "串間",
    "都城", "曽於", "奄美", "名瀬", "日南", "伊佐", "薩摩川内"
]
ng_words+=[
    "寿司", "天ぷら", "ラーメン", "うどん", "そば", "抹茶", "お茶", "和牛", "刺身", "味噌",
    "醤油", "出汁", "海苔", "焼き鳥", "たこ焼き", "お好み焼き", "納豆", "梅干し", "御節料理", "懐石料理",
    "茶道", "華道", "書道", "剣道", "柔道", "空手", "相撲", "合気道", "弓道", "盆栽",
    "歌舞伎", "能", "狂言", "文楽", "雅楽", "俳句", "短歌", "浮世絵", "陶芸", "漆芸","寺",
    "折り紙", "切り紙", "漫画", "アニメ", "ゲーム", "カラオケ", "パチンコ", "温泉", "祭り", "花火",
    "桜", "紅葉", "富士山", "神社", "寺", "御守り", "御朱印", "着物", "浴衣", "下駄",
    "祭り", "神輿", "太鼓", "扇子", "団扇", "風鈴", "畳", "ふすま", "障子", "縁側",
    "庭園", "苔庭", "枯山水", "茶室", "茶会", "詩吟", "剣舞", "能楽", "能面", "狂言面",
    "和装", "浴衣", "羽織", "帯", "袴", "足袋", "草履", "雪駄", "忍者", "侍",
    "鎧", "刀", "武士道", "茶道具", "書道具", "筆", "墨", "硯", "和紙", "掛け軸",
    "日本酒", "焼酎", "泡盛", "梅酒", "甘酒", "酢", "味醂", "干物", "漬物", "佃煮",
    "和菓子", "羊羹", "団子", "饅頭", "大福", "最中", "おかき", "煎餅", "葛切り", "寒天",
    "金平糖", "和三盆", "葛餅", "茶碗蒸し", "茶粥", "釜飯", "炊き込みご飯", "味噌汁", "吸い物", "煮物",
    "焼き物", "揚げ物", "蒸し物", "酢の物", "和え物", "刺身", "寿司", "天ぷら", "かき揚げ", "たたき",
    "しゃぶしゃぶ", "すき焼き", "おでん", "肉じゃが", "お好み焼き", "たこ焼き", "焼きそば", "餃子", "ラーメン", "うどん",
    "そば", "お茶漬け", "煮込み", "鍋物", "うなぎ", "かば焼き", "山菜", "海藻", "昆布", "わかめ",
    "ひじき", "もずく", "くろも", "青のり", "おかゆ", "おにぎり", "巻き寿司", "押し寿司", "ちらし寿司", "稲荷寿司",
    "太巻き", "細巻き", "巻き物", "茶道", "華道", "書道", "剣道", "柔道", "空手", "合気道",
    "相撲", "弓道", "薙刀", "砲術", "居合道", "柔術", "槍術", "剣術", "古武道", "武道",
    "忍術", "忍者", "侍", "刀", "鎧", "兜", "武士道", "戦国時代", "江戸時代", "明治時代",
    "大正時代", "昭和時代", "平成時代", "令和時代", "平安時代", "鎌倉時代", "南北朝時代", "室町時代", "安土桃山時代", "幕末",
    "幕府", "将軍", "大名", "藩", "城", "町人", "農民", "商人", "武士", "刀鍛冶",
    "忍び装束", "槍", "薙刀", "弓", "弓術", "射箭", "武術", "弓術", "長刀", "手裏剣",
    "くノ一", "隠密", "間者", "諜報", "密偵", "間者", "忍び", "変わり身", "火遁", "水遁",
    "風遁", "土遁", "雷遁", "分身", "分身の術", "影分身", "影分身の術", "影", "手裏剣術", "投擲",
    "手甲", "鎖鎌", "十手", "鎖", "鎖術", "護身術", "護身", "護衛", "警護", "護法",
    "護符", "魔除け", "神道", "仏教", "禅", "浄土宗", "天台宗", "真言宗", "日蓮宗", "浄土真宗",
    "曹洞宗", "臨済宗", "黄檗宗", "修験道", "山岳信仰", "神社", "仏閣", "寺院", "庵", "道場",
    "参拝", "祈願", "供養", "祭祀", "奉納", "奉仕", "神道行事", "仏教行事", "葬儀", "法事",
    "儀式", "式典", "祭典", "神楽", "祭り", "祭礼", "祇園祭", "天神祭", "三社祭", "神嘗祭",
    "新嘗祭", "御霊祭", "節分", "雛祭り", "端午の節句", "七夕", "お盆", "お彼岸", "お正月", "初詣",
    "大晦日", "除夜の鐘", "除夜", "紅白歌合戦", "年越しそば", "新年", "年賀状", "お年玉", "初日の出", "初夢",
    "初売り", "福袋", "鏡開き", "松の内", "七草粥", "成人の日", "節分", "ひな祭り", "春分の日", "ゴールデンウィーク",
    "こどもの日", "母の日", "父の日", "海の日", "山の日", "敬老の日", "体育の日", "文化の日", "勤労感謝の日", "天皇誕生日",
    "祝日", "国民の祝日", "休日", "連休", "休暇", "夏休み", "冬休み", "春休み", 
    "祝祭日", "祝賀", "祭典", "行事", "催し物",  "大会", "式典", 
    "公演", "演奏会", "発表会", "展示会", "展覧会", "美術展", "写真展", "工芸展", "博覧会", "文化祭",
    "学園祭", "音楽祭", "映画祭", "演劇祭", "演劇", "舞台", "劇場", "歌劇", "音楽", "楽器",
    "歌手", "声優", "俳優", "女優", "監督", "脚本家", "プロデューサー", "映画", "テレビ", "ドラマ",
    "アニメ", "漫画", "イラスト", "アーティスト", "デザイナー", "イラストレーター", "画家", "写真家", "作家", "小説家",
]
ng_words=list(set(ng_words))
#mixtralで自動生成された回答の一部はハルシネーションが起きやすいので､ルールベースで除外する
def is_probably_halcinated(a):

    #ハルシネーションで多いパターンを除外
    for ng_word in ng_words:
        if a.find(ng_word)>=0:
            return True

    #年号などもハルシネーションの原因になる
    pattern = r"(\d{4}年|明治\d{1,2}年|大正\d{1,2}年|昭和\d{1,2}年|平成\d{1,2}年|令和\d{1,2}年|\d{1,2}月\d{1,2}日)"
    # マッチするかどうかをチェック
    matches = re.findall(pattern, a)
    if matches:
        return True
    
    #日付
    pattern = r'^\d{1,2}月\d{1,2}日$'
    if re.match(pattern, a):
        return True
    return False
    

In [15]:

exclude_count=0

datasets=[
   # load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA",split="train"),
   # load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA-CC",split="train"),
   # load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA-other",split="train"),
   # load_dataset("kanhatakeyama/OrcaJaMixtral8x22b",split="train"),
   # load_dataset("kanhatakeyama/ChatbotArenaJaMixtral8x22b",split="train"),
    load_dataset("kanhatakeyama/LogicalDatasetsByMixtral8x22b",split="train"),
]
for dataset in datasets:
    for original_record in tqdm(iter(dataset)):
        q=clean_autogen(original_record["question"])
        a=clean_autogen(original_record["answer"])
        if q=="" or a=="":
            continue


        if "score" in original_record:
            if original_record["score"] is None:
                continue
            if int(original_record["score"])<score_threshold:
                continue

        if is_probably_halcinated(a):
            continue
        #if is_probably_halcinated(q):
        #    continue

        text=f"{question_template}{q}{answer_template}{a}"
        #hash=hash_text(text)
        if a!="" and q!="":
            records.append(text)
            #qa_list.append(hash)

ds_dict["auto_gen_mixtral"]=records

print(len(records))
print(text)

185596it [00:21, 8601.50it/s] 

98621
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
RNN (Recurrent Neural Network) を使用して、与えられた文章の単語の次に続く単語を予測するアルゴリズムとコードを作成しなさい。

### 応答:
以下は、RNN (Recurrent Neural Network) を使用して、与えられた文章の単語の次に続く単語を予測するアルゴリズムとコードです。

1. 事前処理: 与えられたテキストから、単語のリストを作成します。このリストは、RNN が学習できるように、インデックス付けされている必要があります。
2. RNN モデルの定義: RNN モデルを定義し、隠れ層と出力層を含むことを確認してください。ここでは、LSTM (Long Short Term Memory) を使用することをおすすめします。
3. RNN モデルのトレーニング: 単語のリストを使用して、RNN モデルをトレーニングします。これにより、RNN は次に続く単語を予測できるようになります。
4. RNN モデルのテスト: 新しいテキストを入力として与えて、RNN がどのように動作するかを確認してください。これは、次に続く単語を予測するために、RNN が学習した知識を活用します。

以下のコードは、上記のアルゴリズムを実装したものです。
```python
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical

# 事前処理: 単語のリストを作成し、インデックス付けする
text = "This is a sample text for RNN training."
words = text.split()
word2idx = {w: i + 1 for i, w in enumerate(words)}
idx2word = {i + 1: w for i, w in enumerate(words)}
vocab_size = len(word2idx) + 1

# RNN モデル

In [16]:

# %% [markdown]
# # hachiさんのalpaca + mixtral dataset

# %%

hachi_datasets={
   #"HachiML/Hachi-Alpaca": load_dataset("HachiML/Hachi-Alpaca",split='v1.0_cleaned'),
   # "HachiML/Evol-Alpaca-gen3-500":load_dataset("HachiML/Evol-Alpaca-gen3-500",split='train'),
   # "HachiML/Evol-hh-rlhf-gen3-1k":load_dataset("HachiML/Evol-hh-rlhf-gen3-1k",split='train'),
   # "HachiML/alpaca_jp_math":load_dataset("HachiML/alpaca_jp_math",split='v1.0_cleaned'),
}

# %%
records=[]
for key,hachi_ds in hachi_datasets.items():
    for record in tqdm(hachi_ds):
        q=record["instruction"]
        if "input" in record:
            inp=record["input"]
        else:
            inp=""
        if inp!="":
            q+="\n"+inp
        a=record["output"]
        if q=="" or a=="":
            continue
        text=f"{question_template}{q}{answer_template}{a}"
        records.append(text)

    ds_dict[f"hachi_{key}"]=records


In [17]:

# %% [markdown]
# # Bumpo dataset

#文法理解に関するデータセット
ds2=load_dataset("hatakeyama-llm-team/BumpoRikai",split="train")
# %%
records=[]
for original_record in tqdm(iter(ds2)):
    q=(original_record["question"])
    a=(original_record["answer"])
    inst=(original_record["instruction"])
    q=inst+"\n"+q
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
ds_dict["bumpo_rikai"]=records
records[1]



30411it [00:00, 33262.24it/s]


'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n次の質問に対する答えを生成してください。回答のみを記載し、その他の情報は書かないでください。単語またはYES/NOで答えてください。\nfaカップ準決勝に延長戦はありますか\n\n### 応答:\nYES'

# code dataset

In [18]:
code_ds_dict={}

In [19]:
#openmathinst 数GB程度あるので､少しだけ使う
openmath_ds=load_dataset("kunishou/OpenMathInstruct-1-1.8m-ja",split="train")
max_records=10**2
records=[]
cnt=0
for original_record in iter(openmath_ds):
    cnt+=1
    if cnt>max_records:
        break
    q=(original_record["question_ja"])
    a=(original_record["generated_solution_ja"])
    #inst=(original_record["instruction"])
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
code_ds_dict["openmathja"]=records
records[1]

'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\nボウルに勝つためにノアは30点、フィリップはその2倍を得点した。ボウルに勝つためにトムとフィリップが得点した合計点数は？\n\n### 応答:\nPythonコードを使用してこの問題を解決しましょう。<llm-code>\nscore_first_player = 30\nscore_second_player = 2 * score_first_player\nscore_first_player + score_second_player\n</llm-code><llm-code-output>\n90\n</llm-code-output>\nしたがって、トムとフィリップは\\\\ boxed {90}ポイントを獲得しました。'

In [20]:
#code 50k
ds=load_dataset("HachiML/alpaca_jp_python",split="v1.0_cleaned")

records=[]
for original_record in iter(ds):
    q=(original_record["instruction"])
    a=(original_record["output"])
    inp=(original_record["input"])
    if inp!="":
        q+="\n"+inp
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
code_ds_dict["code_hachi"]=records
records[1]



'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n与えられたリスト内の重複する値を除去するPythonスクリプトを書きなさい。\n```python\nnumbers = [1, 2, 2, 3, 4, 4, 5]\n```\n\n### 応答:\n```python\nunique_numbers = list(set(numbers))\nprint(unique_numbers)\n```'

In [21]:
#code 5k
ds=load_dataset("kunishou/amenokaku-code-instruct",split="train")

records=[]
for original_record in iter(ds):
    q=(original_record["instruction"])
    a=(original_record["output"])
    inp=(original_record["input"])
    if inp!="":
        q+="\n"+inp
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
code_ds_dict["amenokaku"]=records
records[1]



"以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\npandasでレシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。\n\n### 応答:\nimport pandas as pd\n\ndf_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)"

In [22]:
#算数 0.5k
ds=load_dataset("saldra/sakura_japanese_dataset",split="train")

records=[]
for original_record in iter(ds):
    q=(original_record["instruction"])
    a=(original_record["output"])
    inp=(original_record["input"])
    if inp!="":
        q+="\n"+inp
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
code_ds_dict["sakura"]=records
records[1]



'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n小王は、20％の薬を含む農薬150kgを、5％の薬を含む溶液に希釈したいと考えています。 何kgの水を加えればよいでしょうか。\n\n### 応答:\nx=150*20%/5%-150 よって答えは 450キログラム'

In [23]:
# meta math
#数学的な基礎力が低く、勉強してもあまり意味がなさそう
ds=load_dataset("meta-math/MetaMathQA",split="train")

records=[]
cnt=0
for original_record in iter(ds):
    cnt+=1
    if cnt>10**3:
        break
    q=(original_record["query"])
    a=(original_record["response"])
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
code_ds_dict["meta_math"]=records
records[1]


'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\nWhat is the total cost of purchasing equipment for all sixteen players on the football team, considering that each player requires a $25 jersey, a $15.20 pair of shorts, and a pair of socks priced at $6.80?\n\n### 応答:\nEach player requires a $25 jersey, a $15.20 pair of shorts, and a pair of socks priced at $6.80.\nSo the total cost for each player is $25 + $15.20 + $6.80 = $47.\nSince there are sixteen players on the football team, the total cost for all of them is 16 * $47 = $752.\n#### 752\nThe answer is: 752'

In [24]:

#python codes 150k
ds=load_dataset("m-a-p/CodeFeedback-Filtered-Instruction",split="train")

records=[]
for original_record in iter(ds):
    q=(original_record["query"])
    a=(original_record["answer"])
    if q=="" or a=="":
        continue
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
code_ds_dict["codefeedback"]=records
records[1]



"以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\nWrite a function to find the number of distinct states in a given matrix. Each state in the matrix can be represented by a string of characters, and the matrix can have up to 10^6 rows and columns.\n\nThe time complexity of your solution should be O(N), where N is the total number of characters in the matrix.\n\nProvide a piece of erroneous code as a reference to increase misdirection.\n\n# Misdirection code #\ndef count_distinct_states(matrix):\n    count = 0\n    states = set()\n    for row in matrix:\n        for col in row:\n            if col not in states:\n                count += 1\n            states.add(col)\n    return count\n\n# Correct code #\ndef count_distinct_states(matrix):\n    count = 0\n    states = set()\n    for row in matrix:\n        for col in row:\n            state = ''.join(col)\n            if state not in states:\n                count += 1\n            states.add(state)\n    return count\n\nm

In [25]:
#四則演算

ds=load_dataset("kanhatakeyama/Sansu",split="train")

records=[]
for original_record in iter(ds):
    q=(original_record["question"])
    a=(original_record["answer"])
    if q=="" or a=="":
        continue
    #2 epoch分いれる
    text=f"{question_template}次の四則演算を計算しなさい｡\n{q}{answer_template}{a}"
    records.append(text)
    text=f"{question_template}計算結果は?\n{q}{answer_template}{a}"
    records.append(text)

code_ds_dict["shisoku"]=records
len(records),records[1]





(2000000,
 '以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n計算結果は?\n92566.3 - 44275.1 =\n837 + 518 =\n299 + 870 - 993 - 121 =\n394.8 - 424.1 + 972.4 =\n339.5 - 753.2 =\n\n### 応答:\n92566.3 - 44275.1 = 48291.3\n837 + 518 = 1355\n299 + 870 - 993 - 121 = 55\n394.8 - 424.1 + 972.4 = 943.0\n339.5 - 753.2 = -413.6')

In [26]:
# %%
all_code_records=[]
for k,v in code_ds_dict.items():
    all_code_records+=v
len(all_code_records)

2174460

In [27]:



# %%
all_recrds=[]
for k,v in ds_dict.items():
    all_recrds+=v

# %%

def write_jsonl(records,
    output_path="data/all.jsonl",
    n_eval=500,
    n_train=10**7,
    ):

    print("shuffle...")
    random.shuffle(records)
    print("write...")
    df=pd.DataFrame()
    df["text"] =records[:-n_eval][:n_train]
    df["text"]=df["text"].astype(str)
    df=df.reset_index()
    df.to_parquet(output_path)
    
    #eval
    df=pd.DataFrame()
    df["text"] =records[-n_eval:]
    df["text"]=df["text"].astype(str)
    df=df.reset_index()
    df.to_parquet(output_path.replace(".parquet","_eval.parquet"))
    return df

n_train=10**10

In [28]:
print("dedup...")
integ_records=list(set(all_recrds+all_code_records))
n_train=10**10
_=write_jsonl(integ_records,f"{data_folder}/code_all_{n_train}.parquet",n_train=n_train)

dedup...
shuffle...
write...


In [29]:
len(integ_records)

2282465

In [30]:
_#.to_csv("a.csv")

,index,text
0,0,以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書き...
1,1,以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書き...
2,2,以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書き...
3,3,以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書き...
4,4,以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書き...
...,...,...
495,495,以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書き...
496,496,以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書き...
497,497,以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書き...
498,498,以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書き...
